In [1]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data
nnfs.init()

In [2]:
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01* np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases

X, y = spiral_data(samples=100, classes=3)

dense1 = Layer_Dense(2,3)
dense1.forward(X)
print(dense1.output[:5])

[[ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [-1.0475188e-04  1.1395361e-04 -4.7983500e-05]
 [-2.7414842e-04  3.1729150e-04 -8.6921798e-05]
 [-4.2188365e-04  5.2666257e-04 -5.5912682e-05]
 [-5.7707680e-04  7.1401405e-04 -8.9430439e-05]]


ACTIVATION FUNCTION: RELU


In [3]:
import numpy as np
inputs = [ 0, 2, -1, 3.3, -2.7, 1.1, 2.2, -100]
outputs = np.maximum(inputs, 0)
print(outputs)

[0.  2.  0.  3.3 0.  1.1 2.2 0. ]


In [4]:
# ReLU Activation 
class Activation_ReLU:
    # forward pass
    def forward(self, inputs):
        # Calculate output values from input
        self.output = np.maximum(inputs, 0)

In [5]:
from nnfs.datasets import spiral_data
# create data sets 
X, y = spiral_data( samples=100, classes=3)

# Create Dense layer with 2 inputs features and 3 output values
dense1 = Layer_Dense(2, 3)

# Create ReLU Activation to be used with dense layer.
activation1 = Activation_ReLU()

# Make forward pass of our training data through this layer
dense1.forward(X)

# Forward pass through activation function.
# Take in output from previous layer
activation1.forward(dense1.output)

# Let's see output of the first few sample:
print(activation1.output[:5])

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


ACTIVATION SOFTMAX FUNCTION

In [ ]:
## Try These Excercises For Yourself
A = [ [1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]]
print(np.sum(A))

# For One Dimensional along Rows
print('Sum along row:',np.sum(A, axis=0))

# For One Dimensional along Columns
print('Sum along columns:', np.sum(A, axis=1))

#For 2 dimensional
print('In two dimensional along rows: ', np.sum(A, axis=0, keepdims=True))
print(np.sum(A, axis=0, keepdims=True).shape)
#For 2 dimensional
print('In two dimensional along columns: ', np.sum(A, axis=1, keepdims=True))
print(np.sum(A, axis=1, keepdims=True).shape)

78
Sum along row: [15 18 21 24]
Sum along columns: [10 26 42]
In two dimensional along rows:  [[15 18 21 24]]
(1, 4)
In two dimensional along columns:  [[10]
 [26]
 [42]]
(3, 1)


In [7]:
inputs = [ [1, 2, 3, 2.5], 
           [2., 5., -1., 2], 
           [-1.5, 2.7, 3.3, -0.8]
        ]

#Get unnormalized probabilities
exp_values = np.exp( inputs - np.max(inputs, axis=1, keepdims=True))

#Normalize them ton each sample
probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
print(probabilities)
np.sum(probabilities, axis=1)

[[0.06414769 0.17437149 0.47399085 0.28748998]
 [0.04517666 0.90739747 0.00224921 0.04517666]
 [0.00522984 0.34875873 0.63547983 0.0105316 ]]


array([1., 1., 1.])

In [8]:
#SOFTMAX ACTIVATION FUNCTION

class Activation_Softmax:
#Forward pss
 def forward(self, inputs):
  #Get unnormalized probalilities
  exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
  #Normalize them ton each sample
  probabilities = exp_values / np.sum( exp_values, axis=1, keepdims=True)
  self.output = probabilities
 

In [9]:
# Create data sets 
X, y = spiral_data( samples=100, classes=3)

# Create Dense layer with 2 inputs features and 3 output values
dense1 = Layer_Dense(2, 3)
# Create ReLU Activation to be used with dense layer.
activation1 = Activation_ReLU()
# Second dense 
dense2 = Layer_Dense(3, 3)
#Softmax
activation2 = Activation_Softmax()


# Make forward pass of our training data through this layer
dense1.forward(X)
# Forward pass through activation function.
# Take in output from previous layer
activation1.forward(dense1.output)
dense2.forward(activation1.output)
activation2.forward(dense2.output)


# Let's see output of the first few sample:
print(activation2.output[:3])

[[0.33333334 0.33333334 0.33333334]
 [0.33333364 0.3333334  0.3333329 ]
 [0.33333385 0.3333335  0.33333266]]


# ADDING CATEGORICAL ENTROPY LOSS

In [10]:
# LOSS
class Loss:
    # calculate the data and regularization losses
    # given model output and ground truth values
    def calcualtes(self, output, y):
        #Calculate sample loss
        sample_losses = self.forward(output, y)

        # calculate mean loss
        data_loss = np.mean(sample_losses)

        # return loss
        return data_loss

In [11]:
# Cross Entropy Loss
class Loss_CategoricalCrossEntropy(Loss):
    # Forward Pass
    def forward(self, y_pred, y_true):
        # Number of samples in batch
        sample = len(y_pred)
    # Clip data to prevent division by 0
    # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip( y_pred, 1e-7, 1 - 1e-7)
    # Probabilities for target values
    # Only if categorical labels
        if len(y_true.shape) == 1: # one dim
            correct_confidences = y_pred_clipped[
                range(sample),
                y_true
            ]

        # Mask Values - only for ONE-HOT ENCODED
        elif len(y_true.shape) == 2: # 2 dim
            correct_confidences = np.sum(
                y_pred_clipped*y_true,
                axis=1
            )

        # LOSSES
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

In [ ]:
# Create data sets 
X, y = spiral_data( samples=100, classes=3)

# Create Dense layer with 2 inputs features and 3 output values
dense1 = Layer_Dense(2, 3)
# Create ReLU Activation to be used with dense layer.
activation1 = Activation_ReLU()
# Second dense 
dense2 = Layer_Dense(3, 3)
#Softmax
activation2 = Activation_Softmax()

#Create loss function
loss_function = Loss_CategoricalCrossEntropy()


# Make forward pass of our training data through this layer
dense1.forward(X)
# Forward pass through activation function.
# Take in output from previous layer
activation1.forward(dense1.output)
dense2.forward(activation1.output)
activation2.forward(dense2.output)


# Let's see output of the first few sample:
print(activation2.output[:3])

# Perform a forward pass through activation function
# It takes the output of second dense layer here and returns loss
loss = loss_function.calcualtes(activation2.output, y)

# Print Loss function
print("Loss = " , loss)


[[0.33333334 0.33333334 0.33333334]
 [0.33333305 0.33333364 0.33333334]
 [0.3333327  0.333334   0.3333333 ]]
Loss =  1.0986103


In [17]:
# Calculate Accuracy from output of activation2 and targets
# Calculate values along first axis

predictions = np.argmax(activation2.output, axis=1)
if len(y.shape) == 2:
    y = np.argmax(y , axis=1)
    accuracy = np.mean(predictions == y)

# Print accuracy
print("ACCURACY = ", accuracy)

NameError: name 'accuracy' is not defined